In [ ]:
import tensorflow as tf
import numpy as np
from keras.applications.efficientnet import preprocess_input, EfficientNetB5, EfficientNetB4
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications import MobileNetV3Large, MobileNetV3Small
from keras.applications.efficientnet_v2 import EfficientNetV2B0, EfficientNetV2B1, EfficientNetV2B2, EfficientNetV2B3, EfficientNetV2S, EfficientNetV2M, EfficientNetV2L

# Tạo model
from keras.layers import Dense, GlobalAveragePooling2D, Input, Dropout
from keras.models import Model

# Tạo và xử lý data
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

# Hàm loss và optimizer
from keras.losses import CategoricalCrossentropy
from keras.optimizers import Adam

# Callbacks
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
import seaborn as sns
import matplotlib.pyplot as plt

from tabulate import tabulate
# ROC Curve
from sklearn.preprocessing import label_binarize
#Checkpoint and log
from keras.callbacks import CSVLogger
import tempfile
import os
import time
import csv

#Time inference
import cv2

#SVM
from sklearn.svm import SVC

#Ranger

import tensorflow_addons as tfa


In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

In [ ]:
# Xây dựng mô hình
num_classes = 4
IMG_SIZE = 380
batch_size = 8
epochs = 50

inputs = tf.keras.layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
base_model = tf.keras.applications.efficientnet.EfficientNetB4(weights=None, include_top=True, classes=num_classes) 
predictions = base_model(inputs)

model = Model(inputs=inputs, outputs=predictions)

# Đóng băng các lớp của mô hình base
for layer in base_model.layers:
    layer.trainable = True

model.summary()

In [ ]:
# Đường dẫn đến thư mục chứa dữ liệu
data_dir = '/home/ducna/Coding/Data/Data_Grabcut_Use_Augmented/train'
test_dir = '/home/ducna/Coding/Data/Data_Grabcut_Use_Augmented/test'
# Thiết lập các thông số
validation_ratio = 0.1

# Sử dụng ImageDataGenerator để tạo dữ liệu và áp dụng các biến đổi
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=validation_ratio,
    shear_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode='nearest'
)

# Tạo generator cho tập train và tập validation
train_data_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Chỉ sử dụng tập huấn luyện
)

val_data_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Chỉ sử dụng tập validation
)
test_datagen = ImageDataGenerator(rescale=1 / 255.0)
test_data_generator = test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="rgb",
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=False,
    seed=42
)

In [ ]:
#save log
class TimeTrainCSVLogger(tf.keras.callbacks.CSVLogger):
    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()
        super().on_epoch_begin(epoch, logs)
    
    def on_epoch_end(self, epoch, logs=None):
        epoch_end_time = time.time()
        epoch_duration = epoch_end_time - self.epoch_start_time
        logs['time_per_epoch'] = epoch_duration
        super().on_epoch_end(epoch, logs)
        print(f"Epoch {epoch + 1} - Training Time: {epoch_duration:.2f} seconds")
        
class TrainingTimeCallback(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.start_time = time.time()
        super().on_train_begin(logs)

    def on_train_end(self, logs={}):
        end_time = time.time()
        duration = end_time - self.start_time
        super().on_train_end(logs)
        hours = duration // 3600
        minutes = (duration - (hours * 3600)) // 60
        seconds = duration - ((hours * 3600) + (minutes * 60))
        
        msg = f'Total Training Time: {int(hours)} hours, {int(minutes)} minutes, {seconds:.2f} seconds'
        print(msg)

        # Ghi tổng thời gian huấn luyện vào tệp CSV
        with open('train_logger2.csv', mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Total Training Time (seconds)'])
            writer.writerow([duration])
            writer.writerow(['Total Training Time (hours, minutes, seconds)'])
            writer.writerow([hours, minutes, seconds])

csv_logger = TimeTrainCSVLogger('train_logger.csv', append=True, separator='\t')

checkpoint_filepath = "cp.ckpt"
directory_checkpoint = os.path.dirname(checkpoint_filepath)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    verbose=2,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='auto',
    save_best_only=False,
    save_freq='epoch')


In [ ]:
# loss function
loss = CategoricalCrossentropy()

#Optimizer Ranger
radam = tfa.optimizers.RectifiedAdam(learning_rate=1e-3)
ranger = tfa.optimizers.Lookahead(radam, sync_period=6, slow_step_size=0.5)
# optimizer = Adam(learning_rate=0.001)


model.compile(loss=loss, optimizer=ranger, metrics= ['accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

# Tính toán số bước cho tập train và tập validation
steps_per_epoch = train_data_generator.samples // batch_size
validation_steps = val_data_generator.samples // batch_size

history = model.fit(train_data_generator,
          steps_per_epoch=steps_per_epoch,
          validation_steps= validation_steps, 
          batch_size=batch_size, 
          epochs=epochs, 
          validation_data=val_data_generator, 
          callbacks=[model_checkpoint_callback,csv_logger,TrainingTimeCallback()]
          )

In [ ]:
# Đánh giá mô hình trên tệp test
test_loss, test_accuracy, test_recall, test_precision = model.evaluate(test_data_generator)

print(f'Test_loss: {test_loss}')
print(f'Test_accuracy: {test_accuracy}')
print(f'Test_recall: {test_recall}')
print(f'Test_precision: {test_precision}')

with open('Test_data_metrics.txt', 'w') as file:
    file.write(f'Test_loss: {test_loss}\n')
    file.write(f'Test_accuracy: {test_accuracy}\n')
    file.write(f'Test_recall: {test_recall}\n')
    file.write(f'Test_precision: {test_precision}\n')

# Lưu mô hình
model.save('B4_Ranger.h5')

In [ ]:
# Lấy thông tin về độ chính xác trên tập huấn luyện và tập validation
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

# Vẽ đồ thị biểu diễn sự thay đổi của accuracy
plt.plot(range(1, epochs + 1), train_accuracy, label='Training Accuracy')
plt.plot(range(1, epochs + 1), val_accuracy, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy vs Epochs')
plt.legend()
plt.savefig('accuracy_vs_epochs.png') # save before show. If not, it will save a blank image
plt.show()

# Lấy thông tin về loss trên tập huấn luyện và tập validation
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Vẽ đồ thị biểu diễn sự thay đổi của loss
plt.plot(range(1, epochs + 1), train_loss, label='Training Loss')
plt.plot(range(1, epochs + 1), val_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss vs Epochs')
plt.legend()
plt.savefig('loss_vs_epochs.png') # save before show. If not, it will save a blank image
plt.show()

# Extract available metrics keys from the training history
metric_keys = [key for key in history.history.keys() if 'precision' in key]

# Plot the available recall metrics with modified labels
epochs_range = range(1, len(history.history[metric_keys[0]]) + 1)

# Extract the modified labels without "_1" and capitalize them
labels = [key.replace('_1', '').capitalize() for key in metric_keys]

# Plot all recall metrics in one go with modified labels
for key, label in zip(metric_keys, labels):
    plt.plot(epochs_range, history.history[key], label=label)

plt.xlabel('Epochs')
plt.ylabel('Precision vs Epochs')
plt.legend()
plt.savefig('precision_vs_epochs.png') # save before show. If not, it will save a blank image
plt.show()

# Extract available metrics keys from the training history
metric_keys = [key for key in history.history.keys() if 'recall' in key]

# Plot the available recall metrics with modified labels
epochs_range = range(1, len(history.history[metric_keys[0]]) + 1)

# Extract the modified labels without "_1" and capitalize them
labels = [key.replace('_1', '').capitalize() for key in metric_keys]

# Plot all recall metrics in one go with modified labels
for key, label in zip(metric_keys, labels):
    plt.plot(epochs_range, history.history[key], label=label)

plt.xlabel('Epochs')
plt.ylabel('Recall vs Epochs')
plt.legend()
plt.savefig('recall_vs_epochs.png') # save before show. If not, it will save a blank image
plt.show()

# Extract available metrics keys from the training history
metric_keys = [key for key in history.history.keys() if 'recall' in key]

# Plot the available recall metrics with modified labels
epochs_range = range(1, len(history.history[metric_keys[0]]) + 1)

# Extract the modified labels without "_1" and capitalize them
labels = [key.replace('_1', '').capitalize() for key in metric_keys]

# Plot all recall metrics in one go with modified labels
for key, label in zip(metric_keys, labels):
    plt.plot(epochs_range, history.history[key], label=label)

plt.xlabel('Epochs')
plt.ylabel('Recall vs Epochs')
plt.legend()
plt.savefig('recall_vs_epochs.png') # save before show. If not, it will save a blank image
plt.show()

# Dự đoán trên tập kiểm tra và lấy nhãn dự đoán
y_pred = model.predict(test_data_generator)
y_pred_labels = np.argmax(y_pred, axis=1)

# Lấy nhãn thật của tập kiểm tra
y_true_indices = test_data_generator.classes

# Lấy danh sách các lớp từ tên thư mục trong val_data_generator
class_names = list(test_data_generator.class_indices.keys())

# Sử dụng chỉ số thứ tự để lấy tên của các lớp từ danh sách tên lớp
y_true_classes = [class_names[i] for i in y_true_indices]

# Tính toán confusion matrix
cm = confusion_matrix(y_true_indices, y_pred_labels)

# Hiển thị confusion matrix bằng heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.savefig('confusion_matrix.png') # save before show. If not, it will save a blank image
plt.show()

# Tính classification report
class_names = list(test_data_generator.class_indices.keys())
report = classification_report(y_true_indices, y_pred_labels, target_names=class_names, output_dict=True)

# Tạo bảng classification report
report_table = []
for class_name, metrics in report.items():
    if class_name in class_names:
        row = [class_name, metrics['precision'], metrics['recall'], metrics['f1-score']]
        report_table.append(row)

headers = ["Class", "Precision", "Recall", "F1-Score"]
report_text = tabulate(report_table, headers, tablefmt="pretty")
# Lưu bảng classification report vào tệp văn bản
with open('classification_report.txt', 'w') as file:
    file.write(report_text)

# In bảng classification report ra màn hình
print(report_text)

# Chuyển đổi nhãn thành định dạng mã hóa one-hot
y_test_binary = label_binarize(test_data_generator.labels, classes=range(len(class_names)))

# Khởi tạo một từ điển trống để lưu FPR và TPR cho mỗi lớp
fpr = {}
tpr = {}
roc_auc = {}

# Tạo một hình ảnh cho các đường ROC
plt.figure(figsize=(8, 6))

# Duyệt qua từng lớp
for i in range(len(class_names)):
    # Tính toán đường ROC và AUC cho lớp hiện tại
    fpr[i], tpr[i], _ = roc_curve(y_test_binary[:, i], y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

    # Vẽ đường ROC cho lớp hiện tại
    plt.plot(fpr[i], tpr[i], lw=2, label=f'ROC curve for {class_names[i]} (area = {roc_auc[i]:.2f})')

# Cài đặt plot
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.savefig('ROC_curve.png') # save before show. If not, it will save a blank image
plt.show()